In [1]:
import os
os.environ['PYOPENGL_PLATFORM']='osmesa'

import argparse
import pickle
import sys
import cv2
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
# from gta_utils import LIMBS, read_depthmap
# from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm
import glob
# from utils import *
import copy
import argparse

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


import sys
sys.path.append('../')
sys.path.append('../../src')

from benji_prox_dataloader import *
from visualisation import *
from utils import *
import tqdm
from simple_transformer import PoseTransformer
import torch
from torch import nn



numpy: 1.22.3


In [9]:

root_dir = "/cluster/scratch/bdayan/prox_data/"

smplx_model_path='/cluster/home/bdayan/prox/prox/models_smplx_v1_1/models/'
in_frames = 10
pred_frames = 5
batch_size = 8
lr = 0.0001
n_iter = 100

pd = proxDatasetJoints(root_dir=root_dir + '/PROXD', in_frames=in_frames, pred_frames=pred_frames, \
                       output_type='joint_locations', smplx_model_path=smplx_model_path, frame_jump=5)
pdc = DatasetBase(root_dir=root_dir + '/recordings', in_frames=in_frames, pred_frames=pred_frames,
                                             search_prefix='Color', extra_prefix='', frame_jump=5)

pd2 = DatasetBase(root_dir=root_dir + '/PROXD', in_frames=in_frames, pred_frames=pred_frames,
                                             search_prefix='results', extra_prefix='joints_worldnorm.pkl', frame_jump=5)


print(len(pd), len(pdc))
pd.align(pdc)
pdc.align(pd)
pd2.align(pd)
pd.align(pd2)
pdc.align(pd2)
print(len(pd), len(pdc), len(pd2))

# pdi = proxDatasetImages(root_dir="/cluster/scratch/bdayan/prox_data/recordings", in_frames=in_frames, pred_frames=pred_frames)

# dataloader = DataLoader(pd, batch_size=batch_size,
#                         shuffle=True, num_workers=0, collate_fn=my_collate)

2246 2585
2273 2273 2273


In [10]:
pd.output_type = 'joint_locations'
idx, in_joint_locations, pred_joint_locations = pd.__getitem__(500)
pd.output_type = 'raw_pkls'
(idx, (in_joint_fns, in_joint_data), (pred_joint_fns, pred_joint_data)) = pd.__getitem__(500)
in_frames_dicts, in_frames_fns, pred_frames_dicts, pred_frames_fns = pdc.__getitem__(500)

dicts1, fns1, dicts2, fns2 = pd2.__getitem__(500)

In [11]:
def load(fn):
    try:
        with open(fn, 'rb') as file:
            return pickle.load(file)
    except FileNotFoundError:
        return None

joints3 = list(map(load, fns1))

In [13]:
import json
from utils import normalized_joint_locations_world
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


cam2worldfn = root_dir + 'cam2world/' + 'N3OpenArea'
with open(cam2worldfn + '.json') as file:
    cam2world = np.array(json.load(file))
    cam2world = torch.from_numpy(cam2world).float().to(device)


joints = [normalized_joint_locations_world(data_dict, pd.body_model, cam2world) for data_dict in in_joint_data]
def inv(joints):
    joint_locs = joints[:, :25]
    cam_R = cam2world[:3, :3].reshape([3, 3])
    cam_t = cam2world[:3, 3].reshape([1, 3])
    joint_locs = torch.matmul(torch.inverse(cam_R), (joint_locs - cam_t).permute(0, 2, 1)).permute(0, 2, 1)
    return joint_locs

joints2 = [inv(joint) for joint in joints]

In [15]:
joints[0] - joints3[0]

tensor([[[ 0.0000e+00,  0.0000e+00,  2.9802e-08],
         [ 0.0000e+00,  0.0000e+00,  5.9605e-08],
         [ 0.0000e+00,  0.0000e+00,  5.9605e-08],
         [ 0.0000e+00,  0.0000e+00,  5.9605e-08],
         [-1.1921e-07,  0.0000e+00,  5.9605e-08],
         [ 0.0000e+00,  0.0000e+00,  5.9605e-08],
         [ 0.0000e+00,  0.0000e+00,  5.9605e-08],
         [ 0.0000e+00,  0.0000e+00,  1.1921e-07],
         [ 0.0000e+00,  0.0000e+00,  1.1921e-07],
         [ 0.0000e+00,  0.0000e+00,  5.9605e-08],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
         [-5.9605e-08,  0.0000e+00,  2.3842e-07],
         [ 0.0000e+00,  0.0000e+00,  5.9605e-08],
         [ 0.0000e+00,  0.0000e+00,  7.4506e-08],
         [-5.9605e-08,  0.0000e+00,  5.9605e-08],
         [ 0.0000e+00,  0.0000e+00,  1.1921e-07],
         [ 0.0000e+00,  0.0000e+00,  5.9605e-08],
         [ 0.0000e+00,  0.0000e+00,  5.9605e-08],
         [ 0.0000e+00,  0.0000e+00,  5.9605e-08],
         [ 0.0000e+00,  0.0000e+00,  5.9605e-08],


In [16]:
import math

In [19]:
math.ceil(45/5)

9